In [65]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


In [66]:
df = pd.read_csv('news.csv')

In [32]:
df.shape

(6335, 4)

In [33]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [34]:
labels = df.label

In [35]:
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [36]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], labels, test_size = 0.2, random_state = 7)

In [37]:
tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7)

In [38]:
tfidf_train = tfidf.fit_transform(x_train)

In [39]:
tfidf_test = tfidf.transform(x_test)

In [40]:
pac = PassiveAggressiveClassifier(max_iter = 50)

In [41]:
pac.fit(tfidf_train, y_train)

C:\Users\uru\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
              early_stopping=False, fit_intercept=True, loss='hinge',
              max_iter=50, n_iter=None, n_iter_no_change=5, n_jobs=None,
              random_state=None, shuffle=True, tol=None,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [42]:
y_pred = pac.predict(tfidf_test)

In [43]:
score = accuracy_score(y_test, y_pred)

In [44]:
score

0.9273875295974744

In [45]:
confusion_matrix(y_test, y_pred, labels = ['FAKE', 'REAL'])

array([[587,  51],
       [ 41, 588]], dtype=int64)

In [67]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [47]:

df = df[['text', 'label']]

In [48]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.label = le.fit_transform(df.label)

C:\Users\uru\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [68]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [50]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stp_words = stopwords.words('english')
from nltk.stem import PorterStemmer

In [51]:
regex = re.compile('[^a-zA-Z]')
pst = PorterStemmer()

In [52]:
li = []
for sentence in df.text:
    tokens = word_tokenize(sentence)
    temp_li = []
    for word in tokens:
        if (word not in stp_words) & (len(word) > 2):
            word = pst.stem(word)
            word = regex.sub('',word)
            temp_li.append(word)
    temp_li = ' '.join(temp_li)
    li.append(temp_li)
            

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words = 'english', max_df = 0.7)


In [54]:
x = vect.fit_transform(li)

In [55]:
x = pd.DataFrame(x.toarray(), columns = vect.get_feature_names())


In [56]:
y = df['label']

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

In [59]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [60]:
model.fit(x_train, y_train)

C:\Users\uru\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [61]:
pred = model.predict(x_test)

In [62]:
accuracy_score(y_test, pred)

0.8301767676767676

In [63]:
pac.fit(x_train, y_train)
pred = pac.predict(x_test)
accuracy_score(y_test, pred)

C:\Users\uru\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.9330808080808081

In [64]:
confusion_matrix(y_test, pred)

array([[748,  43],
       [ 63, 730]], dtype=int64)